In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import re
from tqdm.notebook import tqdm
import csv

In [ ]:
url = 'https://decisaoimoveis.com.br/compra/?pagination='

urlsAnuncios = [] # Lista que guardará as urls de todos os anuncios

for i in tqdm(range(1, 249)): #1 - 249: páginas de anuncios
    html = urlopen(url + str(i))
    bs = BeautifulSoup(html)
    
    for j in bs.find_all('div', attrs={'class':'item-info col-sm-7'}): # Encontra a url de cada anúncio da página
        urlsAnuncios.append(j.a['href'])

  0%|          | 0/248 [00:00<?, ?it/s]

In [6]:
# Salva as urls em um arquivo .txt
file = open('urls.txt', 'w')

for i in urlsAnuncios:
    file.write(i + '\n')
    
file.close()

In [8]:
# Lê o arquivo .txt com as urls e as armazena em uma lista
file = open('urls.txt', 'r')

urlsAnunciosM = []

for line in file.readlines():
    urlsAnunciosM.append(line.strip())

In [74]:
# Inicializa o header do .csv que armazenará as entradas
csvFile = open('bd.csv', 'w', encoding='utf-8', newline='')
header = ['Código', 'Título', 'Bairro', 'Endereço', 'Área interna', 'Área externa', '# de quartos', '# de banheiros', '# de suítes', '# de vagas', 'IPTU', 'Condomínio', 'Preço']
writer = csv.writer(csvFile)
writer.writerow(header)
csvFile.close()

In [75]:
startPoint = 0 # Ponto de inicio atual para não precisar reiniciar do zero caso encontre algum erro
total = len(urlsAnunciosM)

In [89]:
# Código | Título | Bairro | Endereço | Área interna | Área externa | # de quartos | # de banheiros | # de suítes | # de vagas | IPTU | Condomínio | Preço
for i in tqdm(range(startPoint, total)):
    html
    try:
        html = urlopen(urlsAnunciosM[i])
    except:
        errorFile = open('errorUrls.txt', 'a', newline='')
        errorFile.write(urlsAnunciosM[i] + '\n')
        errorFile.close()
        continue
    bs = BeautifulSoup(html)
    data = []

    # Código
    cod = bs.find('span', {'class':'head-info'}).text
    cod = int(cod.replace('CÓD: ', ''))
    data.append(cod)

    # Título
    titulo = bs.find('div', {'class':'information'}).h2.text
    data.append(titulo)

    # Bairro
    bairro = bs.find_all('span', {'class':'head-info'})[1].text
    bairro = bairro.replace('BAIRRO: ', '')
    data.append(bairro)

    # Endereço
    endereco = bs.find('div', {'class':'uter'}).find_all('script')[1].text
    endereco = re.findall(r'"([^"]*)"', endereco)[0]
    data.append(endereco)


    # Especificações
    specs = bs.find_all('div', {'class':'specs-item'})
    proxSpec = 1

    # Área interna
    try:
        areaI = specs[0].find('div', {'class':'inner-area'}).find_all('span')[1].text
        areaI = float(areaI.replace(' m²', '').replace(',','.'))
        data.append(areaI)
    except:
        areaI = float("nan")
        data.append(areaI)

    # Área externa 
    areaE = ""
    try:
        areaE = specs[1].find('div', {'class':'inner-area'}).find_all('span')[1].text
    except:
        areaE = float("nan")
        data.append(areaE)
    else:
        proxSpec += 1
        areaE = float(areaE.replace(' m²', '').replace(',','.'))
        data.append(areaE)

    # # de quartos
    try:
        quartos = specs[proxSpec].find('span').text
        quartos = int(quartos.replace(' quartos', ''))
        data.append(quartos)
    except:
        quartos = float("nan")
        proxSpec -= 1
        data.append(quartos)

    # # de banheiros
    try:
        banheiros = specs[proxSpec + 1].find('span').text
        banheiros = int(banheiros.replace(' banheiros', ''))
        data.append(banheiros)
    except:
        banheiros = float("nan")
        proxSpec -= 1
        data.append(banheiros)

    # # de suítes
    try:
        suites = specs[proxSpec + 2].find('span').text
        suites = int(suites.replace(' suíte', ''))
        data.append(suites)
    except:
        suites = float("nan")
        proxSpec -= 1
        data.append(suites)

    # # de vagas
    try:    
        vagas = specs[proxSpec + 3].find('span').text
        vagas = int(vagas.replace(' vagas', ''))
        data.append(vagas)
    except:
        vagas = float("nan")
        proxSpec -= 1
        data.append(vagas)

    # IPTU e Condomínio
    specs2 = bs.find('div', {'class':'inline specs'}).find_all('span')

    if (len(specs2) == 2):
        spec1 = specs2[1].text
        if "IPTU" in spec1:
            iptu = float(spec1.replace('IPTU: R$ ', '').replace('.', '').replace(',','.'))
            data.append(iptu)
            cond = float("nan")
            data.append(cond)
        else:
            iptu = float("nan")
            data.append(iptu)
            cond = float(spec1.replace('Condomínio: R$ ', '').replace('.', '').replace(',','.'))
            data.append(cond)
    elif (len(specs2) == 3):
        iptu = specs2[1].text
        iptu = float(iptu.replace('IPTU: R$ ', '').replace('.', '').replace(',','.'))
        data.append(iptu)

        cond = specs2[2].text
        cond = float(cond.replace('Condomínio: R$ ', '').replace('.', '').replace(',','.'))
        data.append(cond)
    else:
        iptu = float("nan")
        data.append(iptu)

        cond = float("nan")
        data.append(cond)

    # Preço
    preco = bs.find('div', {'class':'price'}).h3.text
    preco = float(preco.replace('R$ ', '').replace('.', '').replace(',', '.'))
    data.append(preco)
    
    # Escreve a nova linha no .csv
    csvFile = open('bd.csv', 'a', encoding='utf-8', newline='')
    writer = csv.writer(csvFile)
    writer.writerow(data)
    csvFile.close()
    
    # Atualiza o novo ponto de início
    startPoint += 1

  0%|          | 0/2095 [00:00<?, ?it/s]